<a href="https://colab.research.google.com/github/bryanMontoya/mlTalleres/blob/main/Taller1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Taller 1**
- Johjan Stiven Puerta Castaño
- Brayan Montoya Osorio


In [ ]:
#Sección de importación de librerías.
import numpy as np
import random
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

**Función de coste:**

In [31]:
#Descripción: Función encargada de calcular el error cuadrático medio. Recibe como parámetros la matriz de características, etiquetas y parámetros.
def funcionCoste(features, labels, parametros):          
  funCoste = 0
  parametros = np.array(parametros)  
  for i in range(len(features)):        
    feature = np.append([1], features[i])                   
    funCoste = funCoste + (sum((parametros*feature)) - labels[i])**2    
  funCoste = funCoste/(2*len(features))
  return funCoste

**Gradiente decendente:**

Se calcula el modelo con los parámetros actuales:

In [ ]:
def modelo(Caracter,i, param):     # i --> i-ecima muestra
  m, Tcarac = np.shape(Caracter)
  x = [1]
  for j in range(Tcarac):
    x.append(Caracter[i,j])  # x --> Vector con el valor de una muestra para cada característica
  return np.sum(np.transpose(param)*x)   # Retorna la predicción de la i-ecima muestra y x

La siguiente función calcula la derivada parcial, necesaria para implementar el gradiente decendente:

In [ ]:
def derParc(mod , X, y, m, t, j):   # mod --> Modelo actual (predicción)  X --> Características   y --> etiquetas   m --> N° muestras     t --> parámetros
  aux = np.zeros(shape = m)
  for i in range(m):
    aux[i] = (mod -y[i])*X[i,j]
  return (1/m)*(np.sum(aux))

Se calcula el gradiente haciendo uso de las funciones anteriores. Esta función retorna un vector con los valores de los parámetros dependiendo del número de iteraciones y la velocidad de aprendizaje ingresados por el usuario.

In [ ]:
def gradDec(X_train, y_train, alpha, itera):     # dataSetX --> Características   dataSety --> Etiquetas
                                                              # alpha --> Learning rate    itera --> N° iteraciones
  m, Tcarac = np.shape(X_train)
  t = [random.uniform(-10,10)]
  for _ in range(Tcarac):
    t.append(random.uniform(-10,10))   # Se generan los valores aleatorios iniciales para los parámetros
  
  for _ in range(itera):
    for i in range(Tcarac):
      mode = modelo(X_train, i, t)
      der = derParc(mode, X_train, y_train, m, t, i)
      t[i] = t[i]-alpha*der  
  return t

**Estandarización min-max y Z-score:**

Se implementan dos tipos de estandarización. Para escoger la estandarización que se desee, basta con modificar el valor del parámetro "paramEstandarizacion": 1 para Min-Max y 2 para Z-score, otro para no realizar estandarización.

In [ ]:
#Descripción: Función encargada de dividir entre train y test además de aplicar estandarización a la matriz de características.
def estandarizar(paramEstandarizacion, features, labels, test_size):
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = test_size, random_state = 4)    
    #Estandarización Min-Max.
    if (paramEstandarizacion == 1):
        for feature in range(X_train.shape[1]):
            xx = X_train[: , feature]
            x_min = xx.min()
            x_max = xx.max()        
            for i in range(len(X_train)):
                X_train[i , feature] = (X_train[i , feature] - x_min)/(x_max - x_min)  
            for i in range(len(X_test)):
                X_test[i , feature] = (X_test[i , feature] - x_min)/(x_max - x_min)    
    #Estandarización Z-score.    
    elif (paramEstandarizacion == 2):
        for feature in range(X_train.shape[1]):
            xx = X_train[: , feature]            
            mean = np.mean(xx)
            std = np.std(xx)
            for i in range(len(X_train)):
                X_train[i , feature] = (X_train[i , feature] - mean)/std  
            for i in range(len(X_test)):
                X_test[i , feature] = (X_test[i , feature] - mean)/std    
    return X_train, X_test, y_train, y_test

**Transformación de características:**

Se pueden realizar tres transformaciones, sólo se debe cambiar el parámetro "trans". Si no se desea ninguna tranformación el valor es 0 ,para $X^2$ corresponde el valor 1, para $e^X$ el valor 2 y para $\sqrt{x}$ el valor 3. Si hay un valor negativo dentro de los datos, no se puede realizar la transformación de la raiz cuadrada.

In [ ]:
#Descripción: Función encargada de realizar la transformación de caracteísticas.
def transCarac(dataSetX, trans):
  m, n = np.shape(dataSetX)
  
  if trans == 0:
    aux = dataSetX

  elif trans == 1:
    aux = np.zeros(shape=(m,n))
    for j in range(n):
      for i in range(m):
        aux[i,j] = dataSetX[i,j]**2
    aux = np.concatenate((dataSetX,aux), axis=1)

  elif trans == 2:
    aux = np.zeros(shape=(m,n))
    for j in range(n):
      for i in range(m):
        aux[i,j] = np.exp(dataSetX[i,j])

  elif trans == 3:
    if np.amin(dataSetX) >= 0:
      aux = np.zeros(shape=(m,n))
      for j in range(n):
        for i in range(m):
          aux[i,j] = dataSetX[i,j]**(1/2)

    else:
      aux = "No es posible realizar la transformación. Hay valores negativos."
  return aux

**Implementación del modelo de regresión lineal:**

Se hace uso de las funciones anteriormente creadas, junto con el dataset de diabetes de la librería sklearn. Se pueden modificar hiperparámetros del modelo como la velocidad de aprendizaje, y el número de iteraciones.

In [32]:
diabetes_X, diabetes_y = load_diabetes(return_X_y=True)
transformacion = 0  # Tipo de transformación a realizar: x**2, e^x y sqrt(x).
estandar = 2        # Tipo de estandarización. 1 para min-max, 2 para z-score.
alpha = 0.001       # Velocidad de aprendizaje.
iteraciones = 10000 # Cantidad de iteraciones.

X_transf = transCarac(diabetes_X,trans=transformacion)
X_train, X_test, y_train, y_test = estandarizar(estandar, X_transf, diabetes_y, test_size = 0.2)
t = gradDec(X_train, y_train, alpha, iteraciones)
print("Vector de parámetros calculados:",t)
J = funcionCoste(diabetes_X, diabetes_y, t)
print("Error cuadrático medio:", J)

Vector de parámetros calculados: [169.41251392180143, 21.636609405732145, 443.5905962744573, 362.7211760356833, 176.17778606108521, 136.31194753929617, -313.50435721817865, 353.2163945158, 444.3561670094873, 281.05161715229406, -8.243868815696185]
Error cuadrático medio: 2247.0293395541607


**Regresión lineal sin usar machine learning:**

In [33]:
#Descripción: Función encargada de calcular el vector de parámetros para el conjunto de características sin usar machine learning.
def linearReg(X_train, y_train):    
  X_train_t = np.transpose(X_train)
  parameters = np.dot(np.dot(np.linalg.inv((np.dot(X_train_t, X_train))), X_train_t), y_train)
  return parameters

#Descripción: Función encargada de añadir una columna de unos a la matriz de características.
def addColumns(X_train):
  filas,c = X_train.shape
  xx = np.ones((filas, c + 1)) 
  for fila in range(filas):
    xx[fila] = np.append(1, X_train[fila])
  return xx

In [34]:
print("Vector de parámetros calculados sin usar machine learning: ", linearReg(addColumns(X_train), y_train))

Vector de parámetros calculados sin usar machine learning:  [151.63172805   1.57877308 -13.87492156  23.1625913   17.73916692
 -45.27597611  27.5074988   11.03361289  13.59655962  39.31814162
   1.76796595]
